# --- Day 12: Hot Springs ---
You finally reach the hot springs! You can see steam rising from secluded areas attached to the primary, ornate building.

As you turn to enter, the researcher stops you. "Wait - I thought you were looking for the hot springs, weren't you?" You indicate that this definitely looks like hot springs to you.

"Oh, sorry, common mistake! This is actually the onsen! The hot springs are next door."

You look in the direction the researcher is pointing and suddenly notice the massive metal helixes towering overhead. "This way!"

It only takes you a few more steps to reach the main gate of the massive fenced-off area containing the springs. You go through the gate and into a small administrative building.

"Hello! What brings you to the hot springs today? Sorry they're not very hot right now; we're having a lava shortage at the moment." You ask about the missing machine parts for Desert Island.

"Oh, all of Gear Island is currently offline! Nothing is being manufactured at the moment, not until we get more lava to heat our forges. And our springs. The springs aren't very springy unless they're hot!"

"Say, could you go up and see why the lava stopped flowing? The springs are too cold for normal operation, but we should be able to find one springy enough to launch you up there!"

There's just one problem - many of the springs have fallen into disrepair, so they're not actually sure which springs would even be safe to use! Worse yet, their condition records of which springs are damaged (your puzzle input) are also damaged! You'll need to help them repair the damaged records.

In the giant field just outside, the springs are arranged into rows. For each row, the condition records show every spring and whether it is operational (.) or damaged (#). This is the part of the condition records that is itself damaged; for some springs, it is simply unknown (?) whether the spring is operational or damaged.

However, the engineer that produced the condition records also duplicated some of this information in a different format! After the list of springs for a given row, the size of each contiguous group of damaged springs is listed in the order those groups appear in the row. This list always accounts for every damaged spring, and each number is the entire size of its contiguous group (that is, groups are always separated by at least one operational spring: #### would always be 4, never 2,2).

So, condition records with no unknown spring conditions might look like this:
```
#.#.### 1,1,3
.#...#....###. 1,1,3
.#.###.#.###### 1,3,1,6
####.#...#... 4,1,1
#....######..#####. 1,6,5
.###.##....# 3,2,1
```
However, the condition records are partially damaged; some of the springs' conditions are actually unknown (?). For example:
```
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
```
Equipped with this information, it is your job to figure out how many different arrangements of operational and broken springs fit the given criteria in each row.

In the first line (???.### 1,1,3), there is exactly one way separate groups of one, one, and three broken springs (in that order) can appear in that row: the first three unknown springs must be broken, then operational, then broken (#.#), making the whole row #.#.###.

The second line is more interesting: .??..??...?##. 1,1,3 could be a total of four different arrangements. The last ? must always be broken (to satisfy the final contiguous group of three broken springs), and each ?? must hide exactly one of the two broken springs. (Neither ?? could be both broken springs or they would form a single contiguous group of two; if that were true, the numbers afterward would have been 2,3 instead.) Since each ?? can either be #. or .#, there are four possible arrangements of springs.

The last line is actually consistent with ten different arrangements! Because the first number is 3, the first and second ? must both be . (if either were #, the first number would have to be 4 or higher). However, the remaining run of unknown spring conditions have many different ways they could hold groups of two and one broken springs:
```
?###???????? 3,2,1
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#
```
In this example, the number of possible arrangements for each row is:
```
???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 4 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 1 arrangement
????.######..#####. 1,6,5 - 4 arrangements
?###???????? 3,2,1 - 10 arrangements
```
Adding all of the possible arrangement counts together produces a total of 21 arrangements.

For each row, count all of the different arrangements of operational and broken springs that meet the given criteria. What is the sum of those counts?


## --- Part Two ---
As you look out at the field of springs, you feel like there are way more springs than the condition records list. When you examine the records, you discover that they were actually folded up this whole time!

To unfold the records, on each row, replace the list of spring conditions with five copies of itself (separated by ?) and replace the list of contiguous groups of damaged springs with five copies of itself (separated by ,).

So, this row:
```
.# 1
```
Would become:
```
.#?.#?.#?.#?.# 1,1,1,1,1
```
The first line of the above example would become:
```
???.###????.###????.###????.###????.### 1,1,3,1,1,3,1,1,3,1,1,3,1,1,3
```
In the above example, after unfolding, the number of possible arrangements for some rows is now much larger:
```
???.### 1,1,3 - 1 arrangement
.??..??...?##. 1,1,3 - 16384 arrangements
?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
????.#...#... 4,1,1 - 16 arrangements
????.######..#####. 1,6,5 - 2500 arrangements
?###???????? 3,2,1 - 506250 arrangements
```
After unfolding, adding all of the possible arrangement counts together produces 525152.

Unfold your condition records; what is the new sum of possible arrangement counts?

In [1]:
from pathlib import Path
import os

yr = 2023
d = 12

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = []
  for l in inp.split('\n'):
    lsp = l.split(' ')
    formatted_input.append((lsp[0],
     [int(n.strip()) for n in lsp[1].split(',')]))
  return formatted_input

In [3]:
def check_solution_partial(solution, actual, contigs):
  '''
  Check to see if some prefix of a solution is valid against the
  prefix of a solution. This is a little bit different than checking that
  a full solution is valid because we need to only check that the 
  given contiguous '#' sections in the solution COULD be extended to
  generate a valid solution.
  '''
  assert('?' not in solution)
  for cs, ca in zip(solution, actual[:len(solution)+1]):
    if '#' in [cs, ca] and '.' in [cs, ca]:
      return False
  solution_contigs = [len(s) for s in [x for x in solution.split('.') if len(x) !=0]]
  if len(solution_contigs) > len(contigs):
    return False
  i=0
  for sc, c in zip(solution_contigs, contigs[:len(solution_contigs)]):
    if i!=len(solution_contigs)-1 or solution[-1]=='.':
      if sc!=c:
        return False
    else:
      if sc > c:
        return False
    i+=1
  return True


def check_solution(solution, contigs):
  '''
  Check whether a FULL solution is valid for the given contiguous '#'
  spaces.
  '''
  assert('?' not in solution)
  spring_contigs = [x for x in solution.split('.') if len(x) !=0]
  if len(spring_contigs) != len(contigs):
    return False
  return all([len(s)==c for s, c in zip(spring_contigs, contigs)])


In [4]:
def n_arrangements_naive(record, unfold=False):
  '''
  Just iteratively check all possible combos for '?' spaces
  '''
  from itertools import product

  if unfold:
    record = (record[0]*5, record[1]*5)

  springs = record[0]
  contigs = record[1]
  q_locs = [i for i, x in enumerate(springs) if x=='?']
  q_potents = list(product('.#', repeat=len(q_locs)))
  c = 0
  from copy import deepcopy
  for potent in q_potents:
    solution_potent = list(springs)
    for ql, qp in zip(q_locs, potent):
      solution_potent[ql] = qp
    c += int(check_solution(''.join(solution_potent), contigs))
  return c

In [5]:
def split_query(record, query, depth):
  q_locs = [i for i, x in enumerate(record[0]) if x=='?']
  q = query + record[0][len(query):q_locs[depth]]

  ql, qr = q + '.', q + '#'

  if depth == len(q_locs)-1:
    ql += record[0][q_locs[depth]+1:]
    qr += record[0][q_locs[depth]+1:]

  return ql, qr


def replace_dots(query):
  import re
  return re.sub('\.+', '.', query)


def n_arrangements_memo(record, unfold=False, query='', num_qs=None, depth=0):
  '''
  Recursively check all possible solutions by starting at the leftmost '?' and 
  setting it to either '.' or '#'. Cache results of equivalent queries so that
  we don't have to calculate equivalent queries multiple times.

  Queries are equivalent IFF:
  1) The contiguous spring sections leading up to the rightmost set '?' are congruent
  2) If the most recent '?' was set to '#', then the last set of contiguous '#' contain the same number of '#'s
  3) The query to the right of the most recently set '?' space is congruent

  Alternatively, query equivalence can be defined by a function f(x) which collapses multiple contiguous '.'
  to a single '.', and removes any leading '.'s. Then x and y are equivalent if f(x) == f(y).

  For example, given the initial pattern:

  .???....?#??....??#? [1, 3, 3]

  We will examine the following three sub-patterns:

  A) .#......###.....??#?
  B) ...#...###......??#?
  C) ..#.....###.....??#?

  Each of these three patterns yield 2 possible solutions. Furthermore, the three patterns are equivalent
  because they all collapse to #.###.??#? under the mapping function f(x).
  '''
  from itertools import product
  from time import time
  from tqdm import tqdm




  if unfold:
    record = (((record[0]+'?')*5)[:-1], record[1]*5)


  springs = record[0]
  contigs = record[1]

  record = (springs, contigs)

  if num_qs is None:
    num_qs = sum([x=='?' for x in record[0]])




  valid_dict = {}
  check_dict = {}
  n_arrangements_dict = {}

  def n_arrangements_memo_(record,  query='', depth=0):


    if depth!=num_qs:

      lquery, rquery = split_query(record, query, depth)
        
      if (replace_dots(lquery), depth) not in valid_dict:
        lvalid = check_solution_partial(lquery, springs, contigs)
        valid_dict[(replace_dots(lquery), depth)] = lvalid
      else:
        lvalid = valid_dict[(replace_dots(lquery), depth)]

      if (replace_dots(rquery), depth) not in valid_dict:
        rvalid = check_solution_partial(rquery, springs, contigs)
        valid_dict[(replace_dots(rquery), depth)] = rvalid
      else:
        rvalid = valid_dict[(replace_dots(rquery), depth)]


      lkey = (replace_dots(lquery), depth)
      rkey = (replace_dots(rquery), depth)
      if lvalid:
        if lkey not in n_arrangements_dict:
          lres = n_arrangements_memo_(record, query=lquery, depth=depth+1)
          n_arrangements_dict[lkey] = lres
        else:
          lres = n_arrangements_dict[lkey]
      else:
        lres =  0

      if rvalid:
        if rkey not in n_arrangements_dict:
          rres = n_arrangements_memo_(record, query=rquery, depth=depth+1)
          n_arrangements_dict[rkey] = rres
        else:
          rres = n_arrangements_dict[rkey]
      else:
        rres =  0
      return (lres + rres)
        
    else:
      if (replace_dots(query), depth) in check_dict:
        return check_dict[(replace_dots(query), depth)]
      else:
        res = int(check_solution(query, contigs))
        check_dict[replace_dots(query)] = res
        return res

  return n_arrangements_memo_(record, query='', depth=0)

In [6]:
def sum_of_arrangements(formatted_input, unfold=False):
  '''
  Get the sum of all the possible arrangements in the input.
  If we don't unfold, then use the naive way, but if we have
  to unfold use the memoized way.
  '''
  import numpy as np

  if unfold==False:
    count_fn = n_arrangements_naive
  elif unfold==True:
    count_fn = n_arrangements_memo
  else:
    raise Exception("Invalid Value For 'unfold'")

  from tqdm.notebook import tqdm
  return sum([count_fn(x, unfold=unfold) for x in tqdm(formatted_input)])

In [7]:
import time

t = time.time()

formatted_input = format_input(inp)

print(sum_of_arrangements(formatted_input, unfold=False))
print(sum_of_arrangements(formatted_input, unfold=True))


print('\nRUNTIME: ', time.time()-t)

  0%|          | 0/1000 [00:00<?, ?it/s]

7633


  0%|          | 0/1000 [00:00<?, ?it/s]

23903579139437

RUNTIME:  46.60451340675354
